#Import package

In [ ]:
import numpy as np
from numpy import random
from matplotlib import pyplot as plt 
from skimage.io import imread, imshow
from tqdm import tqdm
import pandas as pd
import math
import copy
import cv2

In [ ]:
!pip install googleDriveFileDownloader #安裝套件語法 !pip install {套件名稱}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from googleDriveFileDownloader import googleDriveFileDownloader
gdownloader = googleDriveFileDownloader()
gdownloader.downloadFile("https://drive.google.com/uc?id=1kwYYWL67O0Dcbx3dvZIfbGg9NiHdyisr&export=download") # 連結到home work 1 data並下載

Download is starting
FILENAME ::: images.zip


0

In [ ]:
!mkdir images
!unzip /content/images.zip -d ./images #解壓縮

串流輸出內容已截斷至最後 5000 行。
  inflating: ./images/images/n02167151/n02167151_15357.JPEG  
  inflating: ./images/images/n02167151/n02167151_1540.JPEG  
  inflating: ./images/images/n02167151/n02167151_15421.JPEG  
  inflating: ./images/images/n02167151/n02167151_15446.JPEG  
  inflating: ./images/images/n02167151/n02167151_1545.JPEG  
  inflating: ./images/images/n02167151/n02167151_15459.JPEG  
  inflating: ./images/images/n02167151/n02167151_15503.JPEG  
  inflating: ./images/images/n02167151/n02167151_15506.JPEG  
  inflating: ./images/images/n02167151/n02167151_15507.JPEG  
  inflating: ./images/images/n02167151/n02167151_15547.JPEG  
  inflating: ./images/images/n02167151/n02167151_15558.JPEG  
  inflating: ./images/images/n02167151/n02167151_15578.JPEG  
  inflating: ./images/images/n02167151/n02167151_15605.JPEG  
  inflating: ./images/images/n02167151/n02167151_1564.JPEG  
  inflating: ./images/images/n02167151/n02167151_15641.JPEG  
  inflating: ./images/images/n02167151/n02167151_156

#Read txt

In [ ]:
train = open("/content/images/train.txt","r")
train_file = train.readlines()
train.close

<function TextIOWrapper.close()>

In [ ]:
test = open("/content/images/test.txt","r")
test_file = test.readlines()
test.close

<function TextIOWrapper.close()>

In [ ]:
val = open("/content/images/val.txt","r")
val_file = val.readlines()
val.close

<function TextIOWrapper.close()>

#Computational graph

##type 1

In [ ]:
w1 = randn(100,128*128)
w2 = randn(50,100)


for l in range(200):
  for i in tqdm(range(len(train_file))):
    (a1,a2) = train_file[i].strip().split()
    img = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(img,(128,128))
    x = image.reshape(128*128,1)
    f1 = w1.dot(x)
    for ii in range(len(f1)):
      if f1[ii] > 0:
        pass
      else:
        f1[ii] = 0
    f = w2.dot(f1)
    y = np.zeros([50,1])
    y[int(a2)] = int(1)
  
    grad_y_pred = f - y
    grad_w2 = grad_y_pred.dot(f1.T)
    grad_f1 = w2.T.dot(grad_y_pred)
    for ll in range(len(f1)):
      if f1[ll] > 0:
        pass
      else:
        grad_f1[ll] = 0

    grad_w1 = f1.dot(x.T)

    w1 -= 0.0001 * grad_w1
    w2 -= 0.0001 * grad_w2
print(l)

In [ ]:
np.save('w1.npy', w1)
np.save('w2.npy', w2)

##type 2

In [ ]:
x_list = []
y_list = []

for l in tqdm(range(len(train_file))):
  (a1,a2) = train_file[l].strip().split()
  filename = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(filename,(64,64))
  img = image.reshape(1,64*64)
  x_list.append(img)
  label = np.zeros([1,50])
  label[0,int(a2)] = int(1)
  y_list.append(label)

100%|██████████| 63325/63325 [00:55<00:00, 1150.99it/s]


In [ ]:
w1 = random.rand(64*64,100)
w2 = random.rand(100,50)
x1 = np.array(x_list)
y1 = np.array(y_list)
x = x1.reshape(63325,4096)
y = y1.reshape(63325,50)
g_f1 = np.empty((63325,100))


for q in tqdm(range(200)):
  f1 = x.dot(w1)
  for i in range(f1.shape[0]):
    for ii in range(f1.shape[1]):
      if f1[i,ii] > 0:
        pass
      else:
        f1[i,ii] = 0
  f = f1.dot(w2)
  loss = np.square(f - y).sum()
  print(q,loss)
  
  grad_y_pred = f - y
  grad_w2 = 2*f1.T.dot(grad_y_pred)
  grad_f1 = grad_y_pred.dot(2*w2.T)
  for l in range(f1.shape[0]):
    dx = np.zeros((100,100))
    for ll in range(f1.shape[1]):
      if f1[l,ll] > 0:
        dx[ll,ll] = 1
      else:
        pass
    g_f1[l] = grad_f1[l].dot(dx)

  grad_w1 = 2*x.T.dot(g_f1)

  w1 -= 0.0001 * grad_w1
  w2 -= 0.0001 * grad_w2

  0%|          | 0/200 [00:00<?, ?it/s]

0 4.784447237127025e+20


  0%|          | 1/200 [00:23<1:16:52, 23.18s/it]

1 63325.0


  1%|          | 2/200 [00:43<1:11:28, 21.66s/it]

...

 99%|█████████▉| 198/200 [1:09:57<00:42, 21.21s/it]

198 63325.0


100%|█████████▉| 199/200 [1:10:20<00:21, 21.68s/it]

199 63325.0


100%|██████████| 200/200 [1:10:41<00:00, 21.21s/it]


In [ ]:
w1

In [ ]:
xtest_list = []
ytest_list = []

for l in tqdm(range(len(test_file))):
  (a1,a2) = test_file[l].strip().split()
  filename = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(filename,(64,64))
  img = image.reshape(1,64*64)
  xtest_list.append(img)
  label = np.zeros([1,50])
  label[0,int(a2)] = int(1)
  ytest_list.append(label)

100%|██████████| 450/450 [00:00<00:00, 820.07it/s]


In [ ]:
x1 = np.array(xtest_list)
y1 = np.array(ytest_list)
x = x1.reshape(len(test_file),4096)
y = y1.reshape(len(test_file),50)
r = 0
w = 0

for ll in range(len(test_file)):
  t1 = x[ll].dot(w1)
  t2 = t1.dot(w2)
  nx = max(t2)
  fx = np.where(t2 == nx)
  fy = np.where(y[ll] == 1)
  if fx == fy:
    r+=1
  else:
    w+=1
  
acc = (r/(r+w))*100
print('accuracy = ',acc ,'%')

accuracy =  2.0 %


In [ ]:
xval_list = []
yval_list = []

for l in tqdm(range(len(val_file))):
  (a1,a2) = val_file[l].strip().split()
  filename = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(filename,(64,64))
  img = image.reshape(1,64*64)
  xval_list.append(img)
  label = np.zeros([1,50])
  label[0,int(a2)] = int(1)
  yval_list.append(label)

100%|██████████| 450/450 [00:00<00:00, 895.55it/s]


In [ ]:
x1 = np.array(xval_list)
y1 = np.array(yval_list)
x = x1.reshape(len(val_file),4096)
y = y1.reshape(len(val_file),50)
r = 0
w = 0

for ll in range(len(val_file)):
  t1 = x[ll].dot(w1)
  t2 = t1.dot(w2)
  nx = max(t2)
  fx = np.where(t2 == nx)
  fy = np.where(y[ll] == 1)
  if fx == fy:
    r+=1
  else:
    w+=1
  
fin = (r/(r+w))*100
print('accuracy = ',fin ,'%')

accuracy =  2.0 %


#LeNet

##Import package

In [ ]:
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
from abc import ABCMeta, abstractmethod

##Class

In [ ]:
# get batch

def get_batch(X, Y, batch_size):   #隨機取訓練資料
    N = len(X)
    i = random.randint(1, N-batch_size)
    return X[i:i+batch_size], Y[i:i+batch_size]


# Make one Hot

def MakeOneHot(Y, D_out):
    N = Y.shape[0]
    Z = np.zeros((N, D_out))
    Z[np.arange(N), Y] = 1
    return Z


# Fully connected Layer

class FC():

    def __init__(self, D_in, D_out):
        #print("Build FC")
        self.cache = None
        #self.W = {'val': np.random.randn(D_in, D_out), 'grad': 0}
        self.W = {'val': np.random.normal(0.0, np.sqrt(2/D_in), (D_in,D_out)), 'grad': 0}  #隨機假設初始權重
        self.b = {'val': np.random.randn(D_out), 'grad': 0}                  #隨機假設bias

    def _forward(self, X):
        #print("FC: _forward")
        out = np.dot(X, self.W['val']) + self.b['val']                    # X*W+b
        self.cache = X
        return out

    def _backward(self, dout):
        #print("FC: _backward")
        X = self.cache
        dX = np.dot(dout, self.W['val'].T).reshape(X.shape)
        self.W['grad'] = np.dot(X.reshape(X.shape[0], np.prod(X.shape[1:])).T, dout)
        self.b['grad'] = np.sum(dout, axis=0)
        #self._update_params()
        return dX

    def _update_params(self, lr=0.001):
        # Update the parameters
        self.W['val'] -= lr*self.W['grad']
        self.b['val'] -= lr*self.b['grad']


# Sigmoid activation Layer

class Sigmoid():

    def __init__(self):
        self.cache = None

    def _forward(self, X):
        h = 1 / (1 + np.exp(-X))
        self.cache = h
        return h

    def _backward(self, dout):
        h = self.cache
        dX = dout*h*(1-h)
        return dX


# Convolution Layer

class Conv():

    def __init__(self, Cin, Cout, F, stride=1, padding=0, bias=True):
        self.Cin = Cin
        self.Cout = Cout
        self.F = F
        self.S = stride
        #self.W = {'val': np.random.randn(Cout, Cin, F, F), 'grad': 0}
        self.W = {'val': np.random.normal(0.0,np.sqrt(2/Cin),(Cout,Cin,F,F)), 'grad': 0} # Xavier Initialization
        self.b = {'val': np.random.randn(Cout), 'grad': 0}
        self.cache = None
        self.pad = padding

    def _forward(self, X):
        X = np.pad(X, ((0,0),(0,0),(self.pad,self.pad),(self.pad,self.pad)), 'constant')
        (N, Cin, H, W) = X.shape
        H_ = H - self.F + 1
        W_ = W - self.F + 1
        Y = np.zeros((N, self.Cout, H_, W_))

        for n in range(N):
            for c in range(self.Cout):
                for h in range(H_):
                    for w in range(W_):
                        Y[n, c, h, w] = np.sum(X[n, :, h:h+self.F, w:w+self.F] * self.W['val'][c, :, :, :]) + self.b['val'][c]

        self.cache = X
        return Y

    def _backward(self, dout):
        # dout (N,Cout,H_,W_)
        # W (Cout, Cin, F, F)
        X = self.cache
        (N, Cin, H, W) = X.shape
        H_ = H - self.F + 1
        W_ = W - self.F + 1
        W_rot = np.rot90(np.rot90(self.W['val']))

        dX = np.zeros(X.shape)
        dW = np.zeros(self.W['val'].shape)
        db = np.zeros(self.b['val'].shape)

        # dW
        for co in range(self.Cout):
            for ci in range(Cin):
                for h in range(self.F):
                    for w in range(self.F):
                        dW[co, ci, h, w] = np.sum(X[:,ci,h:h+H_,w:w+W_] * dout[:,co,:,:])

        # db
        for co in range(self.Cout):
            db[co] = np.sum(dout[:,co,:,:])

        dout_pad = np.pad(dout, ((0,0),(0,0),(self.F,self.F),(self.F,self.F)), 'constant')
        #print("dout_pad.shape: " + str(dout_pad.shape))
        # dX
        for n in range(N):
            for ci in range(Cin):
                for h in range(H):
                    for w in range(W):
                        #print("self.F.shape: %s", self.F)
                        #print("%s, W_rot[:,ci,:,:].shape: %s, dout_pad[n,:,h:h+self.F,w:w+self.F].shape: %s" % ((n,ci,h,w),W_rot[:,ci,:,:].shape, dout_pad[n,:,h:h+self.F,w:w+self.F].shape))
                        dX[n, ci, h, w] = np.sum(W_rot[:,ci,:,:] * dout_pad[n, :, h:h+self.F,w:w+self.F])

        return dX


# Softmax

class Softmax():

    def __init__(self):
        #print("Build Softmax")
        self.cache = None

    def _forward(self, X):
        #print("Softmax: _forward")
        maxes = np.amax(X, axis=1)
        maxes = maxes.reshape(maxes.shape[0], 1)
        Y = np.exp(X - maxes)
        Z = Y / np.sum(Y, axis=1).reshape(Y.shape[0], 1)
        self.cache = (X, Y, Z)
        return Z # distribution

    def _backward(self, dout):
        X, Y, Z = self.cache
        dZ = np.zeros(X.shape)
        dY = np.zeros(X.shape)
        dX = np.zeros(X.shape)
        N = X.shape[0]
        for n in range(N):
            i = np.argmax(Z[n])
            dZ[n,:] = np.diag(Z[n]) - np.outer(Z[n],Z[n])
            M = np.zeros((N,N))
            M[:,i] = 1
            dY[n,:] = np.eye(N) - M
        dX = np.dot(dout,dZ)
        dX = np.dot(dX,dY)
        return dX


# Max pooling

class MaxPool():
    def __init__(self, F, stride):
        self.F = F
        self.S = stride
        self.cache = None

    def _forward(self, X):
        # X: (N, Cin, H, W): maxpool along 3rd, 4th dim
        (N,Cin,H,W) = X.shape
        F = self.F
        W_ = int(float(W)/F)
        H_ = int(float(H)/F)
        Y = np.zeros((N,Cin,W_,H_))
        M = np.zeros(X.shape) # mask
        for n in range(N):
            for cin in range(Cin):
                for w_ in range(W_):
                    for h_ in range(H_):
                        Y[n,cin,w_,h_] = np.max(X[n,cin,F*w_:F*(w_+1),F*h_:F*(h_+1)])
                        i,j = np.unravel_index(X[n,cin,F*w_:F*(w_+1),F*h_:F*(h_+1)].argmax(), (F,F))
                        M[n,cin,F*w_+i,F*h_+j] = 1
        self.cache = M
        return Y

    def _backward(self, dout):
        M = self.cache
        (N,Cin,H,W) = M.shape
        dout = np.array(dout)
        #print("dout.shape: %s, M.shape: %s" % (dout.shape, M.shape))
        dX = np.zeros(M.shape)
        for n in range(N):
            for c in range(Cin):
                #print("(n,c): (%s,%s)" % (n,c))
                dX[n,c,:,:] = dout[n,c,:,:].repeat(2, axis=0).repeat(2, axis=1)
        return dX*M


# Loss

class CrossEntropyLoss():
    def __init__(self):
        pass

    def get(self, Y_pred, Y_true):
        N = Y_pred.shape[0]
        softmax = Softmax()
        prob = softmax._forward(Y_pred)
        loss = NLLLoss(prob, Y_true)
        Y_serial = np.argmax(Y_true, axis=1)
        dout = prob.copy()
        dout[np.arange(N), Y_serial] -= 1
        return loss, dout


#Negative log likelihood loss
def NLLLoss(Y_pred, Y_true):

    loss = 0.0
    N = Y_pred.shape[0]
    M = np.sum(Y_pred*Y_true, axis=1)
    for e in M:
        #print(e)
        if e == 0:
            loss += 500
        else:
            loss += -np.log(e)
    return loss/N



# Net

class Net(metaclass=ABCMeta):
    # Neural network super class

    @abstractmethod
    def __init__(self):
        pass

    @abstractmethod
    def forward(self, X):
        pass

    @abstractmethod
    def backward(self, dout):
        pass

    @abstractmethod
    def get_params(self):
        pass

    @abstractmethod
    def set_params(self, params):
        pass


# LeNet5

class LeNet5(Net):

    def __init__(self):
        self.pool0 = MaxPool(2,2)
        self.conv1 = Conv(1, 6, 5)
        self.Sigm1 = Sigmoid()
        self.pool1 = MaxPool(2,2)
        self.conv2 = Conv(6, 16, 5)
        self.Sigm2 = Sigmoid()
        self.pool2 = MaxPool(2,2)
        self.FC1 = FC(16*5*5, 120)
        self.Sigm3 = Sigmoid()
        self.FC2 = FC(120, 84)
        self.Sigm4 = Sigmoid()
        self.FC3 = FC(84, 50)
        self.Sigm5 = Sigmoid()

        self.p2_shape = None

    def forward(self, X):
        h0 = self.pool0._forward(X)
        h1 = self.conv1._forward(h0)
        a1 = self.Sigm1._forward(h1)
        p1 = self.pool1._forward(a1)
        h2 = self.conv2._forward(p1)
        a2 = self.Sigm2._forward(h2)
        p2 = self.pool2._forward(a2)
        self.p2_shape = p2.shape
        fl = p2.reshape(X.shape[0],-1) # Flatten
        h3 = self.FC1._forward(fl)
        a3 = self.Sigm3._forward(h3)
        h4 = self.FC2._forward(a3)
        a5 = self.Sigm4._forward(h4)
        h5 = self.FC3._forward(a5)
        a5 = self.Sigm5._forward(h5)
        return a5

    def backward(self, dout):
        dout = self.Sigm5._backward(dout)
        dout = self.FC3._backward(dout)
        dout = self.Sigm4._backward(dout)
        dout = self.FC2._backward(dout)
        dout = self.Sigm3._backward(dout)
        dout = self.FC1._backward(dout)
        dout = dout.reshape(self.p2_shape) # reshape
        dout = self.pool2._backward(dout)
        dout = self.Sigm2._backward(dout)
        dout = self.conv2._backward(dout)
        dout = self.pool1._backward(dout)
        dout = self.Sigm1._backward(dout)
        dout = self.conv1._backward(dout)
        dout = self.pool0._backward(dout)

    def get_params(self):
        return [self.conv1.W, self.conv1.b, self.conv2.W, self.conv2.b, self.FC1.W, self.FC1.b, self.FC2.W, self.FC2.b, self.FC3.W, self.FC3.b]

    def set_params(self, params):
        [self.conv1.W, self.conv1.b, self.conv2.W, self.conv2.b, self.FC1.W, self.FC1.b, self.FC2.W, self.FC2.b, self.FC3.W, self.FC3.b] = params


#SGD Momentum

class SGDMomentum():
    def __init__(self, params, lr=0.001, momentum=0.99, reg=0):
        self.l = len(params)
        self.parameters = params
        self.velocities = []
        for param in self.parameters:
            self.velocities.append(np.zeros(param['val'].shape))
        self.lr = lr
        self.rho = momentum
        self.reg = reg

    def step(self):
        for i in range(self.l):
            self.velocities[i] = self.rho*self.velocities[i] + (1-self.rho)*self.parameters[i]['grad']
            self.parameters[i]['val'] -= (self.lr*self.velocities[i] + self.reg*self.parameters[i]['val'])


## Data processing

###train

In [ ]:
x_list = []
y_list = []

for l in tqdm(range(len(train_file))):
  (a1,a2) = train_file[l].strip().split()
  filename = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(filename,(64,64))
  img = image.reshape(1,64,64)
  x_list.append(img)
  y_list.append(int(a2))

100%|██████████| 63325/63325 [00:53<00:00, 1172.72it/s]


###test

In [ ]:
xtest_list = []
ytest_list = []

for l in tqdm(range(len(test_file))):
  (a1,a2) = test_file[l].strip().split()
  filename = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(filename,(64,64))
  img = image.reshape(1,64,64)
  xtest_list.append(img)
  ytest_list.append(int(a2))

100%|██████████| 450/450 [00:00<00:00, 811.22it/s]


###val

In [ ]:
xval_list = []
yval_list = []

for l in tqdm(range(len(val_file))):
  (a1,a2) = val_file[l].strip().split()
  filename = cv2.imread(f'/content/images/{a1}',cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(filename,(64,64))
  img = image.reshape(1,64,64)
  xval_list.append(img)
  yval_list.append(int(a2))

100%|██████████| 450/450 [00:00<00:00, 857.32it/s]


##Training

In [ ]:
#mnist.init()
#X_train, Y_train, X_test, Y_test = load()
X_train = np.array(x_list)
Y_train = np.array(y_list)
X_test = np.array(xtest_list)
Y_test = np.array(ytest_list)
X_val = np.array(xval_list)
Y_val = np.array(yval_list)
X_train, X_test, X_val = X_train/float(255), X_test/float(255), X_val/float(255)
X_train -= np.mean(X_train)
X_test -= np.mean(X_test)
X_val -= np.mean(X_val)

batch_size = 100
D_in = 64*64
D_out = 50
model = LeNet5()


losses = []
optim = SGDMomentum(model.get_params(), lr=0.0001, momentum=0.80, reg=0.00003)
criterion = CrossEntropyLoss()

In [ ]:
# TRAIN
ITER = 650
for i in tqdm(range(ITER)):
	# get batch, make onehot
	X_batch, Y_batch = get_batch(X_train, Y_train, batch_size)
	Y_batch = MakeOneHot(Y_batch, D_out)

	# forward, loss, backward, step
	Y_pred = model.forward(X_batch)
	loss, dout = criterion.get(Y_pred, Y_batch)
	model.backward(dout)
	optim.step()

	if i % 100 == 0:
		print("%s%% iter: %s, loss: %s" % (100*i/ITER,i, loss))
		losses.append(loss)


# save params
weights = model.get_params()
with open("weights.pkl","wb") as f:
	pickle.dump(weights, f)

#draw_losses(losses)



# VAL SET ACC
#Y_pred = model.forward(X_val)
#result = np.argmax(Y_pred, axis=1) - Y_val
#result = list(result)
#print("VAL--> Correct: " + str(result.count(0)) + " out of " + str(X_val.shape[0]) + ", acc=" + str(result.count(0)/X_val.shape[0]))

# TEST SET ACC
Y_pred = model.forward(X_test)
result = np.argmax(Y_pred, axis=1) - Y_test
result = list(result)
print("TEST--> Correct: " + str(result.count(0)) + " out of " + str(X_test.shape[0]) + ", acc=" + str(result.count(0)/X_test.shape[0]))

  0%|          | 1/650 [00:12<2:18:55, 12.84s/it]

0.0% iter: 0, loss: 4.223641875562432


 16%|█▌        | 101/650 [20:24<1:52:52, 12.34s/it]

15.384615384615385% iter: 100, loss: 3.9612619763937893


 31%|███       | 201/650 [40:47<1:32:07, 12.31s/it]

...

76.92307692307692% iter: 500, loss: 4.1150387697464


 92%|█████████▏| 601/650 [2:02:23<09:56, 12.17s/it]

92.3076923076923% iter: 600, loss: 3.882809710086284


100%|██████████| 650/650 [2:12:22<00:00, 12.22s/it]


TEST--> Correct: 9 out of 450, acc=0.02


# Improved LeNet5

##Class

In [ ]:
# Sigmoid

class Sigmoid():
    def __init__(self):
        self.h = None
        self.X = None

    def _forward(self, X):
        h = 1 / (1 + np.exp(-X))
        self.h = h
        self.X = X
        return h*X

    def _backward(self, dout):
        h = self.h
        X = self.X
        dX = dout*(X*h*(1-h)+h)
        return dX


# LeNet5

class LeNet5(Net):

    def __init__(self):
        self.conv0 = Conv(1, 4, 3)
        self.conv1 = Conv(4, 8, 3)
        self.sigm1 = Sigmoid()
        self.pool1 = MaxPool(2,2)
        self.conv2 = Conv(8, 16, 3)
        self.sigm2 = Sigmoid()
        self.pool2 = MaxPool(2,2)
        self.pool3 = MaxPool(2,2)
        self.FC1 = FC(16*7*7, 200)
        self.sigm3 = Sigmoid()
        self.FC2 = FC(200, 84)
        self.sigm4 = Sigmoid()
        self.FC3 = FC(84, 50)
        self.sigm5 = Sigmoid()

        self.p2_shape = None

    def forward(self, X):
        h0 = self.conv0._forward(X)
        h1 = self.conv1._forward(h0)
        a1 = self.sigm1._forward(h1)
        p1 = self.pool1._forward(a1)
        h2 = self.conv2._forward(p1)
        a2 = self.sigm2._forward(h2)
        p2 = self.pool2._forward(a2)
        p3 = self.pool3._forward(p2)
        self.p3_shape = p3.shape
        fl = p3.reshape(X.shape[0],-1) # Flatten
        h3 = self.FC1._forward(fl)
        a3 = self.sigm3._forward(h3)
        h4 = self.FC2._forward(a3)
        a5 = self.sigm4._forward(h4)
        h5 = self.FC3._forward(a5)
        a5 = self.sigm5._forward(h5)
        return a5

    def backward(self, dout):
        dout = self.sigm5._backward(dout)
        dout = self.FC3._backward(dout)
        dout = self.sigm4._backward(dout)
        dout = self.FC2._backward(dout)
        dout = self.sigm3._backward(dout)
        dout = self.FC1._backward(dout)
        dout = dout.reshape(self.p3_shape) # reshape
        dout = self.pool3._backward(dout)
        dout = self.pool2._backward(dout)
        dout = self.sigm2._backward(dout)
        dout = self.conv2._backward(dout)
        dout = self.pool1._backward(dout)
        dout = self.sigm1._backward(dout)
        dout = self.conv1._backward(dout)

    def get_params(self):
        return [self.conv1.W, self.conv1.b, self.conv2.W, self.conv2.b, self.FC1.W, self.FC1.b, self.FC2.W, self.FC2.b, self.FC3.W, self.FC3.b]

    def set_params(self, params):
        [self.conv1.W, self.conv1.b, self.conv2.W, self.conv2.b, self.FC1.W, self.FC1.b, self.FC2.W, self.FC2.b, self.FC3.W, self.FC3.b] = params


##Training

In [ ]:
x_train = np.array(x_list)
y_train = np.array(y_list)
X_test = np.array(xtest_list)
Y_test = np.array(ytest_list)
X_val = np.array(xval_list)
Y_val = np.array(yval_list)
x_train, X_test, X_val = x_train/float(255), X_test/float(255), X_val/float(255)
x_train -= np.mean(x_train)
X_test -= np.mean(X_test)
X_val -= np.mean(X_val)

batch_size = 64
D_in = 64*64
D_out = 50
model = LeNet5()


losses = []
optim = SGDMomentum(model.get_params(), lr=0.0001, momentum=0.80, reg=0.00003)
criterion = CrossEntropyLoss()

In [ ]:
#隨機排列訓練資料集
n = np.arange(len(x_train))
random.shuffle(n)
X_train = x_train[n]
Y_train = y_train[n]

In [ ]:
# TRAIN
ITER = 400
for i in tqdm(range(ITER)):
	# get batch, make onehot
	X_batch, Y_batch = get_batch(X_train, Y_train, batch_size)
	Y_batch = MakeOneHot(Y_batch, D_out)

	# forward, loss, backward, step
	Y_pred = model.forward(X_batch)
	loss, dout = criterion.get(Y_pred, Y_batch)
	model.backward(dout)
	optim.step()

	if i % 100 == 0:
		print("%s%% iter: %s, loss: %s" % (100*i/ITER,i, loss))
		losses.append(loss)


# save params
weights = model.get_params()
with open("weights.pkl","wb") as f:
	pickle.dump(weights, f)

#draw_losses(losses)



# VAL SET ACC
Y_pred = model.forward(X_val)
result = np.argmax(Y_pred, axis=1) - Y_val
result = list(result)
print("VAL--> Correct: " + str(result.count(0)) + " out of " + str(X_val.shape[0]) + ", acc=" + str(result.count(0)/X_val.shape[0]))

# TEST SET ACC
Y_pred = model.forward(X_test)
result = np.argmax(Y_pred, axis=1) - Y_test
result = list(result)
print("TEST--> Correct: " + str(result.count(0)) + " out of " + str(X_test.shape[0]) + ", acc=" + str(result.count(0)/X_test.shape[0]))

  0%|          | 1/400 [00:58<6:26:29, 58.12s/it]

0.0% iter: 0, loss: 44.64385945697879


 25%|██▌       | 101/400 [1:37:32<4:46:13, 57.44s/it]

25.0% iter: 100, loss: 3.947318256630938


 50%|█████     | 201/400 [3:13:48<3:11:45, 57.82s/it]

50.0% iter: 200, loss: 3.9338833390610035


 75%|███████▌  | 301/400 [4:50:06<1:36:33, 58.52s/it]

75.0% iter: 300, loss: 3.933173559236654


100%|██████████| 400/400 [6:27:04<00:00, 58.06s/it]


VAL--> Correct: 17 out of 450, acc=0.03777777777777778
TEST--> Correct: 18 out of 450, acc=0.04
